# Santander Dev Week 2023 (ETL com Python)

**Contexto:** Você é um cientista de dados no Santander e recebeu a tarefa de envolver seus clientes de maneira mais personalizada. Seu objetivo é usar o poder da IA Generativa para criar mensagens de marketing personalizadas que serão entregues a cada cliente.

**Condições do Problema:**

1. Você recebeu uma planilha simples, em formato CSV ('SDW2023.csv'), com uma lista de IDs de usuário do banco:
  ```
  UserID
  1
  2
  3
  4
  5
  ```
2. Seu trabalho é consumir o endpoint `GET https://sdw-2023-prd.up.railway.app/users/{id}` (API da Santander Dev Week 2023) para obter os dados de cada cliente.
3. Depois de obter os dados dos clientes, você vai usar a API do ChatGPT (OpenAI) para gerar uma mensagem de marketing personalizada para cada cliente. Essa mensagem deve enfatizar a importância dos investimentos.
4. Uma vez que a mensagem para cada cliente esteja pronta, você vai enviar essas informações de volta para a API, atualizando a lista de "news" de cada usuário usando o endpoint `PUT https://sdw-2023-prd.up.railway.app/users/{id}`.



In [1]:
# Utilize sua própria URL se quiser ;)
# Repositório da API: https://github.com/digitalinnovationone/santander-dev-week-2023-api
sdw2023_api_url = 'https://sdw-2023-prd.up.railway.app'

547 548 549

## **E**xtract

Extraia a lista de IDs de usuário a partir do arquivo CSV. Para cada ID, faça uma requisição GET para obter os dados do usuário correspondente.

In [ ]:
# @title
'''import pandas as pd

df = pd.read_csv('SDW2023.csv')
user_ids = df['UserID'].tolist()
print(user_ids)'''

In [ ]:
# @title
'''import requests
import json

def get_user(id):
  response = requests.get(f'{sdw2023_api_url}/users/{id}')
  return response.json() if response.status_code == 200 else None

users = [user for id in user_ids if (user := get_user(id)) is not None]
print(json.dumps(users, indent=2))'''

[
  {
    "id": 4,
    "name": "Pyterson",
    "account": {
      "id": 7,
      "number": "00001-1",
      "agency": "0001",
      "balance": 0.0,
      "limit": 500.0
    },
    "card": {
      "id": 4,
      "number": "**** **** **** 1111",
      "limit": 1000.0
    },
    "features": [],
    "news": [
      {
        "id": 9,
        "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
        "description": "Pyterson, invista hoje para garantir um futuro seguro e pr\u00f3spero. Seu futuro agradece!"
      }
    ]
  },
  {
    "id": 5,
    "name": "Pip",
    "account": {
      "id": 8,
      "number": "00002-2",
      "agency": "0001",
      "balance": 0.0,
      "limit": 500.0
    },
    "card": {
      "id": 5,
      "number": "**** **** **** 2222",
      "limit": 1000.0
    },
    "features": [],
    "news": [
      {
        "id": 10,
        "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",


In [2]:
# Extrair os IDs do arquivo CSV.

import pandas as pd

df = pd.read_csv('SDW2023.csv')
user_ids = df['UserID'].tolist()
print(user_ids)

[547, 548, 549]


In [16]:
# Obter os dados de cada ID usando a API da Santander Dev Week 2023.

import requests
import json

def get_user(id):
    response = requests.get(f'{sdw2023_api_url}/users/{id}')
    return response.json() if response.status_code == 200 else None

users = [user for id in user_ids if (user := get_user(id)) is not None]
print(json.dumps(users, indent=2))

[
  {
    "id": 547,
    "name": "Foo",
    "account": {
      "id": 552,
      "number": "11112-1",
      "agency": "0001",
      "balance": 0.0,
      "limit": 500.0
    },
    "card": {
      "id": 521,
      "number": "**** **** **** 1001",
      "limit": 1000.0
    },
    "features": [],
    "news": []
  },
  {
    "id": 548,
    "name": "Bar",
    "account": {
      "id": 553,
      "number": "11112-2",
      "agency": "0001",
      "balance": 0.0,
      "limit": 500.0
    },
    "card": {
      "id": 522,
      "number": "**** **** **** 1002",
      "limit": 1000.0
    },
    "features": [],
    "news": []
  },
  {
    "id": 549,
    "name": "Baz",
    "account": {
      "id": 554,
      "number": "11112-3",
      "agency": "0001",
      "balance": 0.0,
      "limit": 500.0
    },
    "card": {
      "id": 523,
      "number": "**** **** **** 1003",
      "limit": 1000.0
    },
    "features": [],
    "news": []
  }
]


## **T**ransform

Utilize a API do OpenAI GPT-4 para gerar uma mensagem de marketing personalizada para cada usuário.

In [ ]:
# @title
'''!pip install openai
!pip install'''

In [ ]:
# @title
'''# Documentação Oficial da API OpenAI: https://platform.openai.com/docs/api-reference/introduction
# Informações sobre o Período Gratuito: https://help.openai.com/en/articles/4936830

# Para gerar uma API Key:
# 1. Crie uma conta na OpenAI
# 2. Acesse a seção "API Keys"
# 3. Clique em "Create API Key"
# Link direto: https://platform.openai.com/account/api-keys

# Substitua o texto TODO por sua API Key da OpenAI, ela será salva como uma variável de ambiente.
openai_api_key = 'TODO''''

In [ ]:
# @title
'''import openai

openai.api_key = openai_api_key

def generate_ai_news(user):
  completion = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[
      {
          "role": "system",
          "content": "Você é um especialista em markting bancário."
      },
      {
          "role": "user",
          "content": f"Crie uma mensagem para {user['name']} sobre a importância dos investimentos (máximo de 100 caracteres)"
      }
    ]
  )
  return completion.choices[0].message.content.strip('\"')

for user in users:
  news = generate_ai_news(user)
  print(news)
  user['news'].append({
      "icon": "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRfooBexkESIF_57OPqtuiv1qix-efMvhTXXzmPb__HkwEYJxERbYXpZghDUb_-3jr32Yc&usqp=CAU",
      "description": news
  })'''

Pyterson, invista para fazer seu dinheiro crescer. Seu futuro financeiro depende disso!
Pip, investir é o caminho para multiplicar seu dinheiro. Vamos fortalecer seu futuro financeiro!
Pep, investimentos são a chave para o futuro financeiro. Cresça seu dinheiro, não apenas o guarde!


Utilize a API do Bard para gerar uma mensagem de marketing personalizada para cada usuário.

In [4]:
!pip install bardapi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 24.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 51.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.7 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993224 sha256=2a0c226b6f92502fd258a2a637d093cc0feb81e3ffd579cb1c11ffa331e1b9b1
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [5]:
# Pegar a chave "__Secure-1PSID" a partir dos cookies gerados pelo página web do Google Bard e preparar para sua atribuição como token par a utilização da API.

bard_api_key = '__Secure-1PSID'

In [28]:
# Implementar a integração com o Bard.

from bardapi import Bard

bard = Bard(token=bard_api_key)

def generate_ai_news(user):
    response = bard.get_answer(f'Você é um especialista em marketing bancário. Crie uma mensagem para {user["name"]} sobre a importância dos investimentos de no maximo 100 caracteres, sem despedida, utilize ponto final apenas no final da mensagem.')['content']
    posicao_ponto = response.find('.')
    return response[:posicao_ponto+1].replace('*','')

for user in users:
    news = generate_ai_news(user)
    print(news)
    user['news'].append({
      "icon": "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRfooBexkESIF_57OPqtuiv1qix-efMvhTXXzmPb__HkwEYJxERbYXpZghDUb_-3jr32Yc&usqp=CAU",
      "description": news
  })

Oi, Foo!

Invista seu dinheiro e alcance seus objetivos.
Oi, Bar!

Invista seu dinheiro e garanta seu futuro.
Oi, Baz!

Invista seu dinheiro e conquiste a independência financeira.


## **L**oad

Atualize a lista de "news" de cada usuário na API com a nova mensagem gerada.

In [ ]:
# @title
'''def update_user(user):
  response = requests.put(f"{sdw2023_api_url}/users/{user['id']}", json=user)
  return True if response.status_code == 200 else False

for user in users:
  success = update_user(user)
  print(f"User {user['name']} updated? {success}!")'''

User Pyterson updated? True!
User Pip updated? True!
User Pep updated? True!


In [31]:
# Atualizar os usuários na API da Santander Dev Week 2023 com os dados enriquecidos.

def update_user(user):
    response = requests.put(f"{sdw2023_api_url}/users/{user['id']}", json=user)
    return True if response.status_code == 200 else False

for user in users:
    success = update_user(user)
    print(f"User {user['name']} updated? {success}!")

User Foo updated? True!
User Bar updated? True!
User Baz updated? True!
